# KDX 뉴스 데이터 수집

이 노트북은 KDX(한국데이터거래소) 관련 뉴스 데이터를 수집하는 프로세스를 담고 있습니다.

## 목표
- KDX 관련 뉴스 기사를 자동으로 수집
- Nate, MBN 등 다양한 소스에서 데이터 수집
- 수집한 데이터를 구조화된 형태로 저장

In [ ]:
# 필요한 라이브러리 임포트
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta
import json
from dotenv import load_dotenv
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys
sys.setrecursionlimit(100000)  # 너무 깊은 구조 대비

os.environ["APP_ENV"] = "DEV"

app_env = os.getenv("APP_ENV", "DEV")

load_dotenv(f".env.{app_env}")


# 프로젝트 루트 경로 추가 (상대 경로 임포트를 위한 설정)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# 모듈화된 코드 임포트
from src.data.crawler import NewsCrawler
from src.utils.helpers import ensure_directory

## 1. 출력 디렉토리 설정

In [ ]:
# 로우 데이터 저장 경로 설정
RAW_DATA_DIR = os.path.join(project_root, 'data', 'raw')
RAW_DATA_PATH = os.path.join(RAW_DATA_DIR, 'news_articles.csv')
ensure_directory(RAW_DATA_DIR)

# 전처리 데이터 저장 경로 설정
PROCESSED_DATA_DIR = os.path.join(project_root, 'data', 'processed')
PROCESSED_DATA_PATH = os.path.join(RAW_DATA_DIR, 'processed_news_articles.csv')
ensure_directory(PROCESSED_DATA_DIR)

# 서머리 데이터 저장 경로 설정
SUMMARY_DATA_DIR = os.path.join(project_root, 'data', 'summaries')
SUMMARY_DATA_PATH = os.path.join(SUMMARY_DATA_DIR, 'news_summaries1.csv')
ensure_directory(SUMMARY_DATA_DIR)

# 비주얼라이제이션 데이터 저장 경로 설정
VIZ_DATA_DIR = os.path.join(project_root, 'data', 'visualizations')
ensure_directory(VIZ_DATA_DIR)

print(f"로우 데이터 저장공간 {RAW_DATA_DIR}")
print(f"전처리 데이터 저장공간 {PROCESSED_DATA_DIR}")
print(f"서머리 데이터 저장공간 {SUMMARY_DATA_DIR}")
print(f"비주얼라이제이션 데이터 저장공간 {VIZ_DATA_DIR}")

## 2. 크롤러 설정

In [ ]:
# 크롤러 설정 정의
# crawler_config = {
#     'search_keywords': ['KDX', '한국데이터거래소', '데이터거래소', 'AI데이터'],
#     'news_sources': ['mk'],
#     'max_articles_per_source': 15,
#     'date_range_days': 30
# }

# # 설정 내용 확인
# print("크롤러 설정:")
# for key, value in crawler_config.items():
#     print(f"  - {key}: {value}")

## 3. 뉴스 기사 수집 실행

In [ ]:
# 크롤러 초기화
crawler = NewsCrawler()

# 크롤러에 설정 적용 (설정 파일무시)
# crawler.config = crawler_config

# 크롤링 실행
print("크롤링 시작...")
start_time = time.time()

articles_df = crawler.run(RAW_DATA_PATH)

end_time = time.time()
execution_time = end_time - start_time

print(f"크롤링 완료! 소요 시간: {execution_time:.2f}초")
print(f"수집된 기사 수: {len(articles_df)}")

## 4. 수집 결과 확인

In [ ]:
# 데이터프레임 미리보기
articles_df.head()

# 소스별 기사 수 확인
source_counts = articles_df['source'].value_counts()
print("소스별 기사 수:")
print(source_counts)

# 기사 내용 샘플 확인
sample_article = articles_df.iloc[0]
print(f"제목: {sample_article['title']}")
print(f"날짜: {sample_article['date']}")
print(f"소스: {sample_article['source']}")
print(f"URL: {sample_article['url']}")
print("\n내용:")
print(sample_article['content'][:500] + '...' if len(sample_article['content']) > 500 else sample_article['content'])

## 5. 데이터 전처리 실행

In [ ]:
# 데이터 전처리 모듈 임포트
from src.data.preprocessor import NewsPreprocessor

# 전처리기 초기화 및 실행
print("데이터 전처리 시작...")
start_time = time.time()

preprocessor = NewsPreprocessor()
processed_df = preprocessor.run(RAW_DATA_PATH, PROCESSED_DATA_PATH)

end_time = time.time()
execution_time = end_time - start_time

print(f"전처리 완료! 소요 시간: {execution_time:.2f}초")
print(f"전처리된 기사 수: {len(processed_df)}")

# 전처리 결과 미리보기
processed_df.head()

## 6. 요약 생성

In [ ]:
# 요약 모듈 임포트
from src.models.summarizer import GPTSummarizer

# 요약 생성기 초기화 및 실행
print("뉴스 요약 생성 시작...")
start_time = time.time()

summarizer = GPTSummarizer()
summary_df = summarizer.run(PROCESSED_DATA_PATH, SUMMARY_DATA_PATH)


end_time = time.time()
execution_time = end_time - start_time

print(f"요약 생성 완료! 소요 시간: {execution_time:.2f}초")
print(f"요약된 기사 수: {len(summary_df)}")

# 요약 결과 미리보기
summary_df[['title', 'summary']].head()

## 7. 시각화 생성

In [ ]:
# 시각화 모듈 임포트
from src.visualization.plotter import NewsVisualizer

# 시각화 생성기 초기화 및 실행
print("시각화 생성 시작...")
start_time = time.time()

visualizer = NewsVisualizer()
viz_results = visualizer.run(SUMMARY_DATA_PATH, VIZ_DATA_DIR)

end_time = time.time()
execution_time = end_time - start_time

print(f"시각화 생성 완료! 소요 시간: {execution_time:.2f}초")
print(f"생성된 시각화 파일: {viz_results}")

## 8. 주요 시각화 결과 확인

In [ ]:
# 워드 클라우드 표시
wordcloud_path = os.path.join(VIZ_DATA_DIR, 'keywords_wordcloud.png')
if os.path.exists(wordcloud_path):
    from IPython.display import Image
    display(Image(filename=wordcloud_path))
else:
    print("워드 클라우드 이미지를 찾을 수 없습니다.")

# 월별 기사 분포
timeline_path = os.path.join(VIZ_DATA_DIR, 'monthly_articles.png')
if os.path.exists(timeline_path):
    from IPython.display import Image
    display(Image(filename=timeline_path))
else:
    print("월별 기사 분포 이미지를 찾을 수 없습니다.")

# 소스 분포
network_path = os.path.join(VIZ_DATA_DIR, 'source_distribution.png')
if os.path.exists(network_path):
    from IPython.display import Image
    display(Image(filename=network_path))
else:
    print("소스분포 이미지를 찾을 수 없습니다.")

## 9. 요약 및 다음 단계

- KDX 관련 뉴스 데이터를 수집, 전처리, 요약, 시각화하는 전체 파이프라인을 실행했습니다.
- 총 수집된 기사 수: {len(articles_df)}개
- 전처리 및 요약된 기사 수: {len(summary_df)}개
- 주요 키워드: AI, 데이터, 거래소, 플랫폼, 비즈니스 등

### 다음 단계
1. 뉴스 소스 확장 (더 많은 언론사 포함)
2. 감성 분석 추가 (기사의 긍정/부정 분석)
3. 실시간 모니터링 시스템 구축
4. 키워드 트렌드 분석 및 예측 모델 개발

In [ ]:
# 파이프라인 실행 요약 정보 저장
summary_info = {
    "execution_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "articles_collected": len(articles_df),
    "articles_processed": len(processed_df),
    "articles_summarized": len(summary_df),
    "output_files": {
        "raw_data": RAW_DATA_PATH,
        "processed_data": PROCESSED_DATA_PATH,
        "summary_data": SUMMARY_DATA_PATH,
        "visualizations": RAW_DATA_DIR
    }
}

# JSON 형태로 저장
summary_path = os.path.join(RAW_DATA_DIR, 'pipeline_summary.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary_info, f, ensure_ascii=False, indent=4)

print(f"파이프라인 실행 요약 정보가 {summary_path}에 저장되었습니다.")